In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### ith elements of arrays store details about ith Constraint 
### A,B,C are arrays storing values of a,b,c for line a*x + b*y = c
### cond stores inequality for ith equation i.e leq / geq / eq
### Z is tuple of two variables i.e. (3,5) for 3*x + 5*y optimization problem
### o is type of optimization i.e maximize, minimize

def LPP_Solver(A,B,C,cond,Z,o):
  MIN = 0
  MAX = 100 #1e3
  scale = 1
  xmin = MIN
  xmax = MAX

  ## Finding the range of x axis
  for i in range(len(A)):
    if B[i] == 0:
      if A[i] == 0:
        print('Invalid input')
        return 
      if A[i]>0 :
        if cond[i] == 'leq':
          xmax = min(xmax,C[i]/A[i])
        elif  cond[i] == 'geq' :
          xmin = max(xmin,C[i]/A[i])
      else :
        if cond[i] == 'leq':
          xmin = max(xmin,C[i]/A[i])
        elif  cond[i] == 'geq' :
          xmax = min(xmax,C[i]/A[i])

  def nothing():
    print('No feasible Region')
    print('Infeasible Solution')
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.show()

  if xmin>xmax:
    return nothing()

  r = int(xmax - xmin)
  deep = 3
  prec = 1/pow(10,deep)
  lprec = 1
  x = np.linspace(xmin, xmax, 1 + int(r/prec))

  ymax = (x*0) + MAX
  ymin = (x*0) + MIN

  plt.axvline(x = 0, ymin=MIN, ymax=MAX)
  plt.plot(x,ymin)

  gmax = (x*0) + MAX 
  gmin = (x*0) + MIN - 0.5

  extra = -1
  point = -1

  for i in range(len(A)):
    equality = "\{}".format(cond[i]) if cond[i]!='eq' else '='
    operator = '+' if B[i]>0 else '-'
    equation = '${}x {} {}y {} {}$'.format(A[i], operator, abs(B[i]), equality, C[i]) 
    if B[i]!=0:
      y = (C[i] - A[i]*x)/B[i]

      ### To convert the line into our local region with range MAX and MIN
      ### Since x,y >=0 we dont have to worry about x = 0 adn y = 0 line of local region
      ### But we need to choose sufficient MAX such that x = MAX and y = MAX lines will contain the feasible region
      y = np.minimum(y,gmax)
      y = np.maximum(y,gmin)
      plt.plot(x,y,label = equation)

      ### Logic : Convert equation into Y = (c - ax)/b form
      ### 1. If condition is leq then feasible region should be below the line else above
      ### 2. We are taking the b from lhs to rhs so while doing that if b < 0 then condition will be reversed
      if B[i]>0:
        if cond[i]=='leq':
          ymax = np.minimum(ymax,y)
        elif cond[i]=='geq':
          ymin = np.maximum(ymin,y)
        elif extra == -1:
          extra = y
        elif point == -1:
          mask = np.where(extra==y)[0]
          if len(mask)==0:
           return nothing() 
          else:
            point = (x[mask[0]],y[mask[0]])
        else:
          mask = np.where(extra==y)[0]
          if len(mask)==0:
            return nothing()
          elif  point != (x[mask[0]],y[mask[0]]):
            return nothing()
      else:
        if cond[i]=='leq':
          ymin = np.maximum(ymin,y)
        elif cond[i]=='geq':
          ymax = np.minimum(ymax,y)
        elif extra == -1:
          extra = y
        elif point == -1:
          mask = np.where(extra==y)[0]
          if len(mask)==0:
            return nothing()
          else:
            point = (x[mask[0]],y[mask[0]])
        else:
          mask = np.where(extra==y)[0]
          if len(mask)==0:
            return nothing()
          elif  point != (x[mask[0]],y[mask[0]]):
            return nothing() 
    else:
      plt.axvline(x = C[i]/A[i], ymin=MIN, ymax=MAX, label = equation)
    

  plt.xlabel(r'$x$')
  plt.ylabel(r'$y$')

  ### Intersection of ymax and ymin where ymax>ymin will decide feasible region
  I = np.where(ymax>=ymin)[0] ### !!!!!!!!!!
  if len(I)==0:
    return nothing()

  ### Range of x based on intersection
  xmin = max(x[I[0]],xmin)
  xmax = min(x[I[len(I)-1]],xmax) 

  if xmin>xmax:
    return nothing()

  plt.fill_between(x, ymax, ymin, where=ymax>=ymin, color='green', alpha=0.5)  ### !!!!
  plt.xlim((xmin-1,xmax+1))
  plt.ylim((np.min(ymin)-1,np.max(ymax)+1)) ### Range of y based on intersection

  ### Finding extreme points
  ### Logic : p1,p2,p3 are three consequtive points on boundary
  ### If p1 + p2 != 2*p3 then it is extreme point (corner must be there)
  ### To nullify the computational error we will take care of precision (here 0.001) while checking the condition
  ### To overcome the computational error we use the function my_add() for Rounding off and Removing duplicate entries while adding to extreme points

  st = 0  # Becomes one if region is unbounded
  extremes = []

  def my_add(p):
    px,py = round(p[0],deep),round(p[1],deep)
    if len(extremes) == 0:
      extremes.append(p)
    else:
      if abs(extremes[-1][0] - px) > prec*10 or abs(extremes[-1][1] - py) > prec*10:
        extremes.append([px,py])
        if px>=MAX-lprec or py>=MAX-lprec:
          ### If any of the extreme point lies on the infinite line then region becomes unbounded 
          return 1
    return 0

  st |= my_add([x[I[0]],ymax[I[0]]])
  st |= my_add([x[I[0]],ymin[I[0]]])


  n = len(I)
  #print(n,len(np.where(ymax>=MAX-lprec)[0]))
  for id in range(1,n-1):
    i = I[id]
    if abs(2*ymax[i]-ymax[i+1]-ymax[i-1])>prec/10 :
      st |= my_add([x[i],ymax[i]])
    if abs(2*ymin[i]-ymin[i+1]-ymin[i-1])>prec/10 :
      st |= my_add([x[i],ymin[i]])
      
  st |= my_add([x[I[n-1]],ymax[I[n-1]]])
  st |= my_add([x[I[n-1]],ymin[I[n-1]]])

  if st:
    print('Unbounded Feasible Region')
  else:
    print('Bounded Feasible Region')
  
  ### Finding optimal solution
  
  a,b = Z
  P = next(iter(extremes))
  ans = a*P[0] + b*P[1]
  tag = 'Extreme Point'

  for p in extremes:
    local = a*p[0] + b*p[1]
    if o == 'maximize':
      if ans < local:
        ans = local
        P = p
    if o == 'minimize':
      if ans > local:
        ans = local
        P = p
    if p[0]<MAX-lprec and p[1]<MAX-lprec:
        print("Extreme point: ({},{}) Value: {}".format(round(p[0],deep-1),round(p[1],deep-1),round(local,deep-1)))
        plt.plot(p[0],p[1],'go',label=tag)
        tag = '_nolegend_'

  ### Find whether optimal solution is line or a point or unbounded
  ### Logic :  If two extreme points give optimal value then we get a line else only one point

  Infinite = -1
  points = []
  for p in extremes:
    local = a*p[0] + b*p[1]
    if ans == local:
      points.append(p)
      if p[0]>=MAX-lprec or p[1]>=MAX-lprec:
        Infinite = 1
        
  if Infinite == 1:
    print('Unbounded Solution')
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.show()
    return
  
  mp = {'minimize':'Minimum','maximize':'Maximum'}

  if len(points) == 1:
    print('Unique {} Solution'.format(mp[o]))
    print('{} Solution: x = {}, y = {}, value = {}'.format(mp[o],P[0],P[1],round(ans,deep)))
    plt.plot(P[0],P[1],'bs',label='Optimal Solution')
  else:
    print('Infinite Number of solutions')
    x1, y1 = points[0]
    x2, y2 = points[1]
    xd, yd = x2-x1, y2-y1
    print('{} Solution: All points on segment joing ({},{}) and ({},{}) , value = {}'.format(mp[o],round(x1,deep-1),round(y1,deep-1),round(x2,deep-1),round(y2,deep-1),round(ans,deep)))
    if yd == 0:
        x = np.linspace(x1,x2,(x2-x1+1)/prec)
        y = x*0 + y1
        plt.plot(x,y,'bs',label='Optimal Solution')
    elif xd == 0:
        plt.axvline(x1,min(y1,y2),max(y1,y2),color='bs',label='Optimal Solution')
    else:
        x = np.linspace(x1,x2,(x2-x1+1)/prec)
        y = x*(yd/xd) + yd*x1 + y1
        plt.plot(x,y,'bs',label='Optimal Solution')
  plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
  plt.show()

In [ ]:
## Problem given in the Assignment
## Bounded Feasible Region, With unique minimum solution

## o is optimization type
o = 'minimize'

## Z = a*x + b*y is stored as Z = (a,b)
Z = (20,10)

## Line format : a*x + b*y (leq,eq,geq) c
## A[i] stores a 
## B[i] stores b
## C[i] stored c
## cond[i] stores leq/eq/geq

A = [1,3,4]
B = [2,1,3]
C = [40,30,60]
cond = ['leq','geq','geq']

## Solves the LPP
LPP_Solver(A,B,C,cond,Z,o)

In [ ]:
### Bounded Feasible Region, With unique maximum solution

A = [1,3,4]
B = [2,1,3]
C = [40,30,60]
cond = ['leq','geq','geq']
o = 'maximize'
Z = (20,10)

LPP_Solver(A,B,C,cond,Z,o)

In [ ]:
### UnBounded Feasible Region, With unbounded solution

A = [1,3,4]
B = [2,1,3]
C = [40,30,60]
cond = ['geq','geq','geq']
o = 'maximize'
Z = (20,10)

LPP_Solver(A,B,C,cond,Z,o)

In [ ]:
### UnBounded Feasible Region, With unique minimum solution

A = [1,3,4]
B = [2,1,3]
C = [40,30,60]
cond = ['geq','geq','geq']
o = 'minimize'
Z = (20,10)

LPP_Solver(A,B,C,cond,Z,o)

In [ ]:
### Bounded Feasible Region, With unique maximum solution

A = [1,3,4]
B = [2,1,3]
C = [40,30,60]
cond = ['leq','leq','leq']
o = 'maximize'
Z = (20,10)

LPP_Solver(A,B,C,cond,Z,o)

In [ ]:
### InFeasible Region
A = [1,3,4]
B = [2,1,3]
C = [40,30,60]
cond = ['geq','geq','leq']
o = 'maximize'
Z = (20,10)

LPP_Solver(A,B,C,cond,Z,o)

In [ ]:
A = [0,1,0,1]
B = [1,0,1,0]
C = [10,10,30,30]
cond = ['geq','geq','leq','leq']
o = 'minimize'
Z = (3,4)

LPP_Solver(A,B,C,cond,Z,o)